<a href="https://colab.research.google.com/github/Evavanrooijen/AfricanGDP/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time

In [0]:
def CRDW(Y, i, tao=0.4):
  """This function tests for cointegration
      Args:
        i (int): index of the country we are modeling
        tao (float): critical value (default = 0.4)

      Returns:
        JH (array): An array with the regressors (incl. self)

  """
  N, T = Y.shape
  JH=Y[i][1:]
  for j in range(N):
      if j!=i:
        y = Y[i]
        x = Y[j]
        x = sm.add_constant(x)

        model = sm.OLS(y, x)
        results = model.fit()
        CRDW_j = stats.stattools.durbin_watson(results.resid)
        if CRDW_j > tao:
          JH = np.vstack((JH, Y[j][1:]))
  assert JH.shape[0]>0 # test if JH contains atleast self
  
  return JH

In [0]:
def max_min_correlations(Y, i, kn=4, kp=4):
    """Feature selection based on pairwise correlation
    Args:
        Y_growth (array): matrix of levels (Y_growth)
        i (int): index of the country we are modeling
        tao (float): critical value (default = 0.4)

    Returns:
        JH (array): An array with the regressors (incl. self)

    """
    N, T = Y.shape
    Y_growth = np.vstack([growth_rate(row) for row in Y])
    assert round(np.mean(Y_growth)) == (alpha+1)/2

    corr_i = np.zeros(N)
    for j in range(N):
      corr_i[j] = pearsonr(Y_growth[i], Y_growth[j])[0]
    
    pos = Y_growth[np.argpartition(corr_i, -(kp+1))[-(kp+1):-1]]
    #pos = Y_growth[corr_i.argsort()[(kp+1):-1]]
    assert pos.shape == (kp, T-1)
    neg = Y_growth[corr_i.argsort()[:kn]]
    assert neg.shape == (kn, T-1)
    #neg = Y_growth[np.argpartition(correlation(7), -5)[-5:-1]]
    return pos, neg

In [0]:
def define_parameters(Y, i):
  JH = CRDW(Y, i)
  pos, neg = max_min_correlations(Y, i)

  params = Parameters()
  rank = JH.shape[0]
  for r in range(1, rank):
    params.add('beta'+str(r), value = np.random.normal(-1, 1), max=0)

  params.add('mu', value = np.random.normal(0, 1))
  params.add('alpha_self', value = np.random.normal(0, 1))
  params.add('alpha_pos', value = np.random.normal(0, 1))
  params.add('alpha_neg', value = np.random.normal(0, 1))
  params.add('theta_pos', value = np.random.normal(0, 1))
  params.add('theta_neg', value = np.random.normal(0, 1))
  params.add('gamma', value = np.random.normal(0, 1))

  params.pretty_print()
  
  return JH, pos, neg, params, rank

In [0]:
# still depending on i

def residual(params, pos, neg, JH, rank):
  # rank = JH.shape[0] # test if this works when rank = 1 NOPE

  mu = params['mu']
  alpha_self = params['alpha_self']
  alpha_pos = params['alpha_pos']
  alpha_neg = params['alpha_neg']
  theta_pos = params['theta_pos']
  theta_neg = params['theta_neg']
  gamma = params['gamma']

  beta = np.ones(rank)
  for r in range(1, rank):
    beta[r] = params['beta'+str(r)]

  assert beta[0] == 1
  #assert np.sum(beta>0) == 1
  assert beta.size == rank
      
  alpha_i_j_p = np.array([alpha_pos, alpha_pos*theta_pos, alpha_pos*(theta_pos**2), alpha_pos*(theta_pos**3)])
  alpha_i_j_n = np.array([alpha_neg, alpha_neg*theta_neg, alpha_neg*(theta_neg**2), alpha_neg*(theta_neg**3)])
  
  assert alpha_i_j_p.size == 4
  assert alpha_i_j_n.size == 4

  # TO-DO growth rate t-1, alpha_self ??
  # correlation = alpha_self * growth_rate(Y) + alpha_i_j_n.dot(neg) + alpha_i_j_p.dot(pos)
  correlation = alpha_i_j_n.dot(neg) + alpha_i_j_p.dot(pos)

  cointegration = gamma * (beta.dot(JH))

  model = cointegration + correlation + mu

  return growth_rate(Y[3]) - model

In [0]:
def function(pos, neg, JH, gamma, beta, mu):
  rank = JH.shape[0] # test if this works when rank = 1 NOPE

  beta = np.ones(rank)
  for r in range(1, rank):
    beta[r] = params['beta'+str(r)]

  assert beta[0] == 1
  assert np.sum(beta>0) == 1
  assert beta.size == rank
      
  alpha_i_j_p = np.ones(4)
  alpha_i_j_n = np.ones(4)
  
  assert alpha_i_j_p.size == 4
  assert alpha_i_j_n.size == 4

  # TO-DO growth rate t-1, alpha_self ??
  # correlation = alpha_self * growth_rate(Y) + alpha_i_j_n.dot(neg) + alpha_i_j_p.dot(pos)
  correlation = alpha_i_j_n.dot(neg) + alpha_i_j_p.dot(pos)

  cointegration = gamma * (beta.dot(JH))

  return cointegration + correlation + mu

In [0]:
# should return scalara value t+1

def fit_and_predict(JH, pos, neg, params, rank):
  start = time.time()
  out = minimize(residual, params, args=(pos, neg, JH), method='leastsq', maxfev=1000000)
  print(out.message)
  print('Estimation took: {} seconds'.format(time.time() - start))

  mu_hat = out.params['mu'].value
  gamma_hat = out.params['gamma'].value
  output = function(pos, neg, JH, mu_hat, [1, -0.4, -0.2], 0.9)

  beta = np.ones(rank)
  for r in range(1, rank):
    beta[r] = out.params['beta'+str(r)].value

  assert beta.size == rank
  assert beta[0] == 1
  assert np.sum(beta > 0) == 1

  prediction = function(pos, neg, JH, mu_hat, beta, gamma_hat)
  
  return prediction

In [0]:
def predict():
  return function(pos, neg, JH, mu_hat, beta, gamma_hat)

In [0]:
def RMSPE(pred, truth):
  return 
